In [1]:
from ckks_package.ckks import *

In [2]:
# Configuration

N = 2**14  # Ring degree
n = 2**3  # Number of slots
L_boot = 19  # Maximal level during bootstrapping
q0 = 2**29  # Smallest modulus
p = 2**23  # Scaling factor outside of bootstrapping
delta = 2**33  # Scaling factor during bootstrapping

CKKS.config(N, n, L_boot, q0, p, delta)
CKKS.key_gen()
CKKS.config_bootstrap(CKKS.sk, d=7, r=8, log_radix=1)

In [ ]:
# Estimating the security (in number of bits)

CKKS.get_security(check_primal_hybrid=False)

47.81

In [4]:
# Creating two random complex vectors

complex_vectors = [
    np.array([randint(-3, 3) + 1j * randint(-3, 3) for _ in range(n)])
    for _ in range(2)
]
complex_vectors

[array([-1.+1.j, -3.+0.j,  3.+1.j,  2.-3.j, -3.+0.j,  1.+3.j, -1.+3.j,
         3.+2.j]),
 array([-0.-2.j,  2.+2.j,  3.+1.j,  3.+3.j,  1.+0.j, -2.+1.j,  0.+1.j,
        -2.-2.j])]

In [5]:
# Encoding as polynomials

plaintext_polys = [CKKS.encode(z) for z in complex_vectors]
plaintext_polys

[1048576 + 2231668X^1024 - 332425X^2048 + 7099044X^3072 - 1482910X^4096 + 11007805X^5120 - 13700306X^6144 - 12097131X^7168 + 7340032X^8192 + 1121782X^9216 + 4677577X^10240 + 1282382X^11264 + 5931642X^12288 - 364488X^13312 + 5674852X^14336 + 2885599X^15360 mod(2^673),
 5242880 + 3824736X^1024 - 1203818X^2048 - 13858766X^3072 - 741455X^4096 - 6085833X^5120 - 6448880X^6144 + 169795X^7168 + 4194304X^8192 + 5600200X^9216 - 2906274X^10240 - 1504129X^11264 - 5190186X^12288 + 1653391X^13312 + 1868668X^14336 - 8724862X^15360 mod(2^673)]

In [6]:
# Encrypting

ciphertexts = [CKKS.enc_poly_with_sk(pt, CKKS.sk) for pt in plaintext_polys]
ciphertexts

[A CKKS ciphertext with degree N = 2^14 and modulus q = (2^29) * (2^23)^28 (level 28 out of 28).,
 A CKKS ciphertext with degree N = 2^14 and modulus q = (2^29) * (2^23)^28 (level 28 out of 28).]

In [8]:
# Homomorphic addition

ct_add = ciphertexts[0] + ciphertexts[1]
pt_add = ct_add.dec_to_poly(CKKS.sk)
print(ct_add)
print(CKKS.decode(pt_add))

A CKKS ciphertext with degree N = 2^14 and modulus q = (2^29) * (2^23)^28 (level 28 out of 28).
[-1.0000023 -1.00000225e+00j -0.99999853+2.00000018e+00j
  6.00000031+2.00000146e+00j  5.00000057-5.80018162e-07j
 -1.99999866+7.11162745e-07j -1.00000028+4.00000089e+00j
 -0.99999987+4.00000300e+00j  1.00000257-5.40285050e-07j]


In [9]:
# Homomorphic multiplication

ct_mult = ciphertexts[0] @ ciphertexts[1]
pt_mult = ct_mult.dec_to_poly(CKKS.sk)
print(ct_mult)
print(CKKS.decode(pt_mult))

A CKKS ciphertext with degree N = 2^14 and modulus q = (2^29) * (2^23)^27 (level 27 out of 28).
[ 2.00000238+2.00000262e+00j -5.99999545-5.99999838e+00j
  7.99999956+6.00000463e+00j 14.99999719-3.00000169e+00j
 -3.00000174-3.59727167e-06j -4.99999659-4.99999979e+00j
 -3.00000421-9.99999437e-01j -1.99999447-9.99999767e+00j]


In [10]:
# Bootstrapping

ct = ciphertexts[0] % q0
ct_boot = ct.bootstrap()
pt_boot = ct_boot.dec_to_poly(CKKS.sk)
print(CKKS.decode(pt_boot))
print(ct.get_precision(ct_boot, CKKS.sk))

[-0.99817861+1.00567672e+00j -2.98961819+2.25862493e-03j
  2.98997545+9.94744408e-01j  1.99415927-2.99194228e+00j
 -2.99451061+3.38011085e-03j  1.00183546+2.99031066e+00j
 -0.99566992+2.99024905e+00j  2.99197186+1.99661376e+00j]
11.78
